In [ ]:
import platform
import matplotlib.pyplot as plt
print(platform.python_version())

# Todo List

- [x] add calculated year to the metadata
- [ ] make one chart per year
- [x] make sure all the data files are getting loaded

In [ ]:
# Get ye data
import glob
import os

data_files = glob.glob(os.path.expanduser('~\\Data\\*.json'))

for file in data_files:
	print(f"{file}", os.path.getsize(file))



In [ ]:
import pandas as pd

data_frames = []
for file in data_files:

	f = open(file, encoding='utf-8')
	df_temp = pd.read_json(f)
	data_frames.append(df_temp)

df = pd.concat(data_frames, ignore_index=True)

In [ ]:
import numpy as np
df['count'] = np.zeros(len(df))



#### Pie Chart

In [ ]:

dfp = df
# dfp['mile'] = [m.get('title') if m else None for m in dfp.milestone]
dfp['mile'] = [m.get('title') if hasattr(m, 'get') else None for m in dfp['milestone']]


In [ ]:
closed_dates = pd.to_datetime(df['closedAt'], format='ISO8601')
dfp['year'] = closed_dates.dt.year

dfp.drop(columns=['closedAt','milestone', 'url'], inplace=True)
print(dfp)

In [ ]:

mile_counts = dfp.groupby('mile')['title'].nunique()
dfp['count'] = dfp['mile'].map(mile_counts)
# dfp.drop(columns=['title'], inplace=True)
dfp['mile'] = df['mile'].drop_duplicates()
dfp.dropna(inplace=True)
dfp = dfp[dfp['count'] > 5]
print(dfp)

In [ ]:
mile_counts = df.groupby('mile')['count'].sum()
mile_counts = mile_counts.loc[lambda x: x > 5]


print(mile_counts)

In [ ]:

plt.pie(mile_counts, labels=mile_counts.index, autopct='%0.1f%%')
plt.title(f"Tasks by Milestone")
plt.axis('equal')
plt.show()

In [ ]:
type(mile_counts)

#### Plot of closed issues by month and milestone for one project


In [ ]:
closed_months = pd.to_datetime(df['closedAt'], format='ISO8601')
df['closed_month'] = closed_months.dt.month
df.drop(columns=['closedAt'], inplace=True)

In [ ]:
df['mile'] = [d.get('title') if d else None for d in df.milestone]

In [ ]:

dfg = df.groupby(['mile', 'closed_month']).count().reset_index()
dfg.drop(columns=['milestone', 'title', 'url'], inplace=True)


In [ ]:
exclude_from_chart = [
	'Job Aid Ideas']

for ex in exclude_from_chart:
	dfg = dfg[dfg['mile'] != ex]


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()

In [ ]:
plt.figure(figsize=(10, 6))
sns.pie(x='closed_month', y='count', hue='mile', data=dfg)
plt.pie(dfg, )
plt.xlabel('Closed Month')
plt.ylabel('ClosedAt')
plt.title('Stacked Line Chart: Milestone Effort by Issue Close Date')
plt.xticks(rotation=45)
plt.legend(title='Mile', loc='upper left')
plt.show()